In [2]:
%matplotlib inline

In [3]:
import matplotlib
matplotlib.use('TkAgg')

In [4]:
import matplotlib.pyplot as plt
#plt.switch_backend('agg')
import matplotlib.ticker as ticker
import numpy as np

In [5]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [6]:
SOS_token = 0
EOS_token = 1


class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS"}
        self.n_words = 2  # Count SOS and EOS

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

In [7]:
# Turn a Unicode string to plain ASCII, thanks to
# https://stackoverflow.com/a/518232/2809427
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

# Lowercase, trim, and remove non-letter characters


def normalizeString(s):
    s = unicodeToAscii(s.lower().strip())
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
    return s

In [8]:
def readLangs(lang1, lang2, reverse=False):
    print("Reading lines...")

    # Read the file and split into lines
    lines = open('data/%s-%s.txt' % (lang1, lang2), encoding='utf-8').\
        read().strip().split('\n')

    # Split every line into pairs and normalize
    pairs = [[normalizeString(s) for s in l.split('\t')] for l in lines]

    # Reverse pairs, make Lang instances
    if reverse:
        pairs = [list(reversed(p)) for p in pairs]
        input_lang = Lang(lang2)
        output_lang = Lang(lang1)
    else:
        input_lang = Lang(lang1)
        output_lang = Lang(lang2)

    return input_lang, output_lang, pairs

In [9]:
MAX_LENGTH = 45

eng_prefixes = (
    "i am ", "i m ",
    "he is", "he s ",
    "she is", "she s ",
    "you are", "you re ",
    "we are", "we re ",
    "they are", "they re "
)


def filterPair(p):
    return len(p[0].split(' ')) < MAX_LENGTH and \
        len(p[1].split(' ')) < MAX_LENGTH #and \
       # p[1].startswith(eng_prefixes)


def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]

In [10]:
def prepareData(lang1, lang2, reverse=False):
    input_lang, output_lang, pairs = readLangs(lang1, lang2, reverse)
    print("Read %s sentence pairs" % len(pairs))
    pairs = filterPairs(pairs)
    print("Trimmed to %s sentence pairs" % len(pairs))
    print("Counting words...")
    for pair in pairs:
        input_lang.addSentence(pair[0])
        output_lang.addSentence(pair[1])
    print("Counted words:")
    print(input_lang.name, input_lang.n_words)
    print(output_lang.name, output_lang.n_words)
    return input_lang, output_lang, pairs


input_lang, output_lang, pairs = prepareData('dar', 'eng', True)
print(random.choice(pairs))

Reading lines...
Read 19679 sentence pairs
Trimmed to 19679 sentence pairs
Counting words...
Counted words:
eng 6531
dar 14863
['the whole ball of wax', ' bb o tben']


In [11]:
pairs2 = random.sample(pairs,len(pairs))
pairs_train = pairs2[:17000]
pairs_test = pairs2[17000:]

In [12]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output = embedded
        output, hidden = self.gru(output, hidden)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [13]:
class AttnDecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, dropout_p=0.1, max_length=MAX_LENGTH):
        super(AttnDecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.dropout_p = dropout_p
        self.max_length = max_length

        self.embedding = nn.Embedding(self.output_size, self.hidden_size)
        self.attn = nn.Linear(self.hidden_size * 2, self.max_length)
        self.attn_combine = nn.Linear(self.hidden_size * 2, self.hidden_size)
        self.dropout = nn.Dropout(self.dropout_p)
        self.gru = nn.GRU(self.hidden_size, self.hidden_size)
        self.out = nn.Linear(self.hidden_size, self.output_size)

    def forward(self, input, hidden, encoder_outputs):
        embedded = self.embedding(input).view(1, 1, -1)
        embedded = self.dropout(embedded)

        attn_weights = F.softmax(
            self.attn(torch.cat((embedded[0], hidden[0]), 1)), dim=1)
        attn_applied = torch.bmm(attn_weights.unsqueeze(0),
                                 encoder_outputs.unsqueeze(0))

        output = torch.cat((embedded[0], attn_applied[0]), 1)
        output = self.attn_combine(output).unsqueeze(0)

        output = F.relu(output)
        output, hidden = self.gru(output, hidden)

        output = F.log_softmax(self.out(output[0]), dim=1)
        return output, hidden, attn_weights

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [14]:
def indexesFromSentence(lang, sentence):
    return [lang.word2index[word] for word in sentence.split(' ')]


def tensorFromSentence(lang, sentence):
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)


def tensorsFromPair(pair):
    input_tensor = tensorFromSentence(input_lang, pair[0])
    target_tensor = tensorFromSentence(output_lang, pair[1])
    return (input_tensor, target_tensor)

In [15]:
teacher_forcing_ratio = 0.5


def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, max_length=MAX_LENGTH):
    encoder_hidden = encoder.initHidden()

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)

    encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

    loss = 0

    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(
            input_tensor[ei], encoder_hidden)
        encoder_outputs[ei] = encoder_output[0, 0]

    decoder_input = torch.tensor([[SOS_token]], device=device)

    decoder_hidden = encoder_hidden

    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    if use_teacher_forcing:
        # Teacher forcing: Feed the target as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            loss += criterion(decoder_output, target_tensor[di])
            decoder_input = target_tensor[di]  # Teacher forcing

    else:
        # Without teacher forcing: use its own predictions as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            topv, topi = decoder_output.topk(1)
            decoder_input = topi.squeeze().detach()  # detach from history as input

            loss += criterion(decoder_output, target_tensor[di])
            if decoder_input.item() == EOS_token:
                break

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length

In [16]:
import time
import math


def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

In [17]:
def trainIters(encoder, decoder, n_iters, print_every=1000, plot_every=100, learning_rate=0.01):
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every

    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)
    training_pairs = [tensorsFromPair(random.choice(pairs))
                      for i in range(n_iters)]
    criterion = nn.NLLLoss()

    for iter in range(1, n_iters + 1):
        training_pair = training_pairs[iter - 1]
        input_tensor = training_pair[0]
        target_tensor = training_pair[1]

        loss = train(input_tensor, target_tensor, encoder,
                     decoder, encoder_optimizer, decoder_optimizer, criterion)
        print_loss_total += loss
        plot_loss_total += loss

        if iter % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, iter / n_iters),
                                         iter, iter / n_iters * 100, print_loss_avg))

        if iter % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0

    #showPlot(plot_losses)

In [18]:
## PB
import matplotlib.pyplot as plt
plt.switch_backend('agg')
import matplotlib.ticker as ticker
import numpy as np


def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()
    # this locator puts ticks at regular intervals
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)

In [19]:
def evaluate(encoder, decoder, sentence, max_length=MAX_LENGTH):
    with torch.no_grad():
        input_tensor = tensorFromSentence(input_lang, sentence)
        input_length = input_tensor.size()[0]
        encoder_hidden = encoder.initHidden()

        encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[ei],
                                                     encoder_hidden)
            encoder_outputs[ei] += encoder_output[0, 0]

        decoder_input = torch.tensor([[SOS_token]], device=device)  # SOS

        decoder_hidden = encoder_hidden

        decoded_words = []
        decoder_attentions = torch.zeros(max_length, max_length)

        for di in range(max_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            decoder_attentions[di] = decoder_attention.data
            topv, topi = decoder_output.data.topk(1)
            if topi.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            else:
                decoded_words.append(output_lang.index2word[topi.item()])

            decoder_input = topi.squeeze().detach()

        return decoded_words, decoder_attentions[:di + 1]

In [20]:
def evaluateRandomly(encoder, decoder, n=10):
    for i in range(n):
        pair = random.choice(pairs)
        print('>', pair[0])
        print('=', pair[1])
        output_words, attentions = evaluate(encoder, decoder, pair[0])
        output_sentence = ' '.join(output_words)
        print('<', output_sentence)
        print('')

In [ ]:
hidden_size = 256
encoder1 = EncoderRNN(input_lang.n_words, hidden_size).to(device)
attn_decoder1 = AttnDecoderRNN(hidden_size, output_lang.n_words, dropout_p=0.1).to(device)

trainIters(encoder1, attn_decoder1, 200000, print_every=1000)

2m 14s (- 447m 3s) (1000 0%) 5.3853
4m 28s (- 443m 40s) (2000 1%) 5.1997
6m 31s (- 428m 49s) (3000 1%) 5.1456
8m 40s (- 425m 1s) (4000 2%) 5.2519
10m 52s (- 423m 53s) (5000 2%) 5.1969
13m 8s (- 424m 56s) (6000 3%) 5.2348
15m 17s (- 421m 26s) (7000 3%) 5.2002
17m 28s (- 419m 16s) (8000 4%) 5.1814
19m 35s (- 415m 57s) (9000 4%) 5.1660
21m 40s (- 411m 57s) (10000 5%) 5.1290
23m 55s (- 410m 56s) (11000 5%) 5.1277
26m 19s (- 412m 18s) (12000 6%) 5.0826
28m 33s (- 410m 48s) (13000 6%) 5.0671
30m 45s (- 408m 33s) (14000 7%) 5.0518
32m 59s (- 406m 52s) (15000 7%) 5.0550
35m 14s (- 405m 18s) (16000 8%) 5.0407
37m 31s (- 404m 0s) (17000 8%) 5.0148
39m 47s (- 402m 20s) (18000 9%) 5.0129
42m 1s (- 400m 19s) (19000 9%) 4.9237
44m 17s (- 398m 35s) (20000 10%) 4.9073
46m 32s (- 396m 40s) (21000 10%) 4.9093
48m 49s (- 395m 4s) (22000 11%) 4.9056
51m 3s (- 392m 54s) (23000 11%) 4.8572
53m 21s (- 391m 17s) (24000 12%) 4.8897
55m 34s (- 388m 58s) (25000 12%) 4.7207
57m 52s (- 387m 20s) (26000 13%) 4.7901

In [20]:
encoder1 = torch.load('encoder2.pt')
attn_decoder1 = torch.load('attn_decoder2.pt')

In [21]:
trainIters(encoder1, attn_decoder1, 80000, print_every=1000)

2m 29s (- 196m 48s) (1000 1%) 2.6934
5m 2s (- 196m 45s) (2000 2%) 2.7321
7m 37s (- 195m 49s) (3000 3%) 2.7411
10m 5s (- 191m 42s) (4000 5%) 2.6271
12m 27s (- 186m 56s) (5000 6%) 2.6690
14m 55s (- 184m 1s) (6000 7%) 2.5716
17m 21s (- 180m 58s) (7000 8%) 2.6250
19m 44s (- 177m 36s) (8000 10%) 2.5957
22m 12s (- 175m 9s) (9000 11%) 2.6031
24m 37s (- 172m 21s) (10000 12%) 2.6587
32m 55s (- 206m 28s) (11000 13%) 2.5259
35m 36s (- 201m 48s) (12000 15%) 2.5823
38m 37s (- 199m 5s) (13000 16%) 2.4438
41m 35s (- 196m 4s) (14000 17%) 2.5348
44m 14s (- 191m 43s) (15000 18%) 2.5184
46m 47s (- 187m 10s) (16000 20%) 2.5560
49m 9s (- 182m 11s) (17000 21%) 2.4614
51m 39s (- 177m 56s) (18000 22%) 2.4624
54m 5s (- 173m 39s) (19000 23%) 2.3874
58m 29s (- 175m 29s) (20000 25%) 2.4547
60m 59s (- 171m 21s) (21000 26%) 2.3952
63m 22s (- 167m 6s) (22000 27%) 2.3986
66m 39s (- 165m 11s) (23000 28%) 2.4464
69m 28s (- 162m 5s) (24000 30%) 2.3834
72m 3s (- 158m 31s) (25000 31%) 2.3233
74m 34s (- 154m 52s) (26000 32

In [22]:
torch.save(encoder1, 'encoder21.pt')
torch.save(attn_decoder1, 'attn_decoder21.pt')

/Users/arthurclaude/opt/anaconda3/lib/python3.7/site-packages/torch/serialization.py:402: UserWarning: Couldn't retrieve source code for container of type EncoderRNN. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/Users/arthurclaude/opt/anaconda3/lib/python3.7/site-packages/torch/serialization.py:402: UserWarning: Couldn't retrieve source code for container of type AttnDecoderRNN. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


In [24]:
trainIters(encoder1, attn_decoder1, 50000, print_every=1000)

2m 31s (- 123m 23s) (1000 2%) 1.6679
5m 0s (- 120m 2s) (2000 4%) 1.6817
7m 31s (- 118m 0s) (3000 6%) 1.6963
9m 57s (- 114m 26s) (4000 8%) 1.6856
12m 26s (- 111m 57s) (5000 10%) 1.7041
14m 52s (- 109m 4s) (6000 12%) 1.7659
17m 18s (- 106m 22s) (7000 14%) 1.6975
19m 56s (- 104m 43s) (8000 16%) 1.6195
22m 24s (- 102m 6s) (9000 18%) 1.6508
24m 48s (- 99m 15s) (10000 20%) 1.6690
27m 12s (- 96m 28s) (11000 22%) 1.6611
29m 37s (- 93m 49s) (12000 24%) 1.5446
32m 5s (- 91m 19s) (13000 26%) 1.6312
34m 24s (- 88m 29s) (14000 28%) 1.5630
36m 54s (- 86m 6s) (15000 30%) 1.6733
39m 19s (- 83m 32s) (16000 32%) 1.6098
41m 46s (- 81m 5s) (17000 34%) 1.6120
44m 17s (- 78m 43s) (18000 36%) 1.6116
46m 48s (- 76m 22s) (19000 38%) 1.6085
49m 21s (- 74m 2s) (20000 40%) 1.6054
51m 45s (- 71m 28s) (21000 42%) 1.5401
54m 17s (- 69m 6s) (22000 44%) 1.5537
56m 49s (- 66m 42s) (23000 46%) 1.5539
59m 18s (- 64m 14s) (24000 48%) 1.5261
61m 39s (- 61m 39s) (25000 50%) 1.5051
64m 4s (- 59m 8s) (26000 52%) 1.5190
66m 29

In [25]:
torch.save(encoder1, 'encoder21.pt')
torch.save(attn_decoder1, 'attn_decoder21.pt')

In [27]:
trainIters(encoder1, attn_decoder1, 50000, print_every=1000)

2m 42s (- 132m 51s) (1000 2%) 1.4227
5m 18s (- 127m 19s) (2000 4%) 1.3950
7m 49s (- 122m 40s) (3000 6%) 1.3568
10m 20s (- 118m 51s) (4000 8%) 1.3838
12m 51s (- 115m 45s) (5000 10%) 1.4212
15m 27s (- 113m 22s) (6000 12%) 1.4659
18m 2s (- 110m 49s) (7000 14%) 1.3848
20m 32s (- 107m 48s) (8000 16%) 1.3956
22m 55s (- 104m 27s) (9000 18%) 1.3533
25m 23s (- 101m 34s) (10000 20%) 1.3647
27m 51s (- 98m 46s) (11000 22%) 1.3361
30m 16s (- 95m 53s) (12000 24%) 1.3888
32m 41s (- 93m 2s) (13000 26%) 1.3008
35m 10s (- 90m 28s) (14000 28%) 1.3782
37m 37s (- 87m 47s) (15000 30%) 1.3178
40m 3s (- 85m 8s) (16000 32%) 1.3619
42m 29s (- 82m 28s) (17000 34%) 1.3300
44m 55s (- 79m 51s) (18000 36%) 1.3730
47m 20s (- 77m 14s) (19000 38%) 1.2917
49m 47s (- 74m 41s) (20000 40%) 1.3221
52m 13s (- 72m 7s) (21000 42%) 1.3652
54m 42s (- 69m 37s) (22000 44%) 1.3465
57m 13s (- 67m 10s) (23000 46%) 1.3066
59m 39s (- 64m 37s) (24000 48%) 1.2848
62m 5s (- 62m 5s) (25000 50%) 1.3443
64m 31s (- 59m 33s) (26000 52%) 1.3556

In [28]:
#total: 300k iterations
torch.save(encoder1, 'encoder21.pt')
torch.save(attn_decoder1, 'attn_decoder21.pt')

In [21]:
enc = torch.load('encoder21.pt')
dec = torch.load('attn_decoder21.pt')

In [ ]:
evaluate(enc, dec, "")[0][0]